In [3]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
pre_fix = '/data/darknet/python/'

image_dir = pre_fix + '/croped_manufacture_images/'
xml_dir = pre_fix + '/croped_manufacture_xmls/'

#generated_image_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_motorbike_bicycle/VOC2007/JPEGImages/'
#generated_xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_motorbike_bicycle/VOC2007/Annotations/'
generated_xml_dir = xml_dir
generated_image_dir = image_dir

#xml_dir = '/data/darknet/python/' + time_str + '_xml_expand_obj/' 
#xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_whgk/VOC2007/Annotations/'
#image_dir = '/data/darknet/python/' + time_str + '_image_expand_obj/'
#image_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_whgk/VOC2007/JPEGImages/'
generated_image_dir = image_dir
#'/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_GuAn/VOC2007/JPEGImages/'
generated_xml_dir = xml_dir
#'/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_GuAn/VOC2007/Annotations/'

#os.system('mkdir -p ' + xml_dir)
#os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir

def get_obj_from_xml(xml_name):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]


def get_fliped_bbox(bbox, img_w):
    return (img_w-bbox[1], img_w-bbox[0], bbox[2], bbox[3])

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
    
def generate_new_xmlobj(xmlobj_old, new_bbox):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text
    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text
    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(int(new_bbox[0]))
    tag_ymin.text = str(int(new_bbox[2]))
    tag_xmax.text = str(int(new_bbox[1]))
    tag_ymax.text = str(int(new_bbox[3]))
    return element_object

def generate_etree(etree_old):
    root_old = etree_old.getroot()    
    root = ET.Element('annotation')
    root.append(root_old.find('folder'))
    root.append(root_old.find('filename'))
    root.append(root_old.find('source'))
    root.append(root_old.find('size'))
    root.append(root_old.find('segmented'))    
    tree = ET.ElementTree(root)    
    return tree

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    

In [4]:
seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()
print(len(seed_xml_names))
for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
        print(img_name)
        print(xml_name)
        
        tree=ET.parse(xml_name)
        new_tree = generate_etree(tree)
        img = cv2.imread(img_name)
        print(img.shape)
        
        objs = get_obj_from_xml(xml_name)
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            new_bbox = get_fliped_bbox(bbox, img.shape[1])
            #print(bbox)
            #print(new_bbox)
            new_obj_element = generate_new_xmlobj(obj_element, new_bbox)
            new_tree = insert_to_xml(new_obj_element, new_tree)
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        #new_tree.write(GENE_XML_DIR + 'fliped_' + time_str + xml_name.split('/')[-1])
        new_tree.write(GENE_XML_DIR + 'fliped_'  + xml_name.split('/')[-1])
        h_flip = cv2.flip(img, 1)
        #cv2.imwrite(GENE_IMG_DIR + 'fliped_' + time_str + img_name.split('/')[-1], h_flip)
        cv2.imwrite(GENE_IMG_DIR + 'fliped_'  + img_name.split('/')[-1], h_flip)

1437
/data/darknet/python//croped_manufacture_images/20190603164006355473_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164006355473_00006005300.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164006526067_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164006526067_00006005381.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164006693973_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164006693973_00006005389.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164006856065_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164006856065_00006005610.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164007012044_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164007012044_00006005633.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164007168679_00006005710

(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164014361900_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164014361900_00006005381.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164014520638_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164014520638_00006005389.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164014706090_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164014706090_00006005610.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164014857667_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164014857667_00006005633.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164015016536_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164015016536_00006005710.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164015179175_0

(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164021997948_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164021997948_00006005300.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164022178518_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164022178518_00006005381.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164022340626_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164022340626_00006005389.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164022507777_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164022507777_00006005610.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164022662520_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164022662520_00006005633.xml
(900, 1600, 3)
/data/darknet/python//croped_manufacture_images/20190603164022823577_0

/data/darknet/python//croped_manufacture_images/20190603164029918696_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164029918696_00006005381.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164030083599_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164030083599_00006005389.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164030242067_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164030242067_00006005610.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164030396537_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164030396537_00006005633.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164030552840_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164030552840_00006005710.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164030705146_00006005300.jpg


(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164037105843_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164037105843_00006005633.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164037264123_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164037264123_00006005710.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164037425084_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164037425084_00006005300.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164037587347_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164037587347_00006005381.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164037750580_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164037750580_00006005389.xml
(830, 1476, 3)
/data/darknet/python//croped_manufacture_images/20190603164037908536_0

...

(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164318332032_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164318332032_00006005389.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164318499229_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164318499229_00006005610.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164318654947_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164318654947_00006005633.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164318821970_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164318821970_00006005710.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164319011129_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164319011129_00006005300.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164319182922_0000600

/data/darknet/python//croped_manufacture_images/20190603164326635454_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164326635454_00006005633.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164326802385_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164326802385_00006005710.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164326966764_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164326966764_00006005300.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164327139554_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164327139554_00006005381.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164327306177_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164327306177_00006005389.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164327477782_00006005610.jpg
/data

(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164335753830_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164335753830_00006005710.xml
(490, 872, 3)
/data/darknet/python//croped_manufacture_images/20190603164335906856_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164335906856_00006005300.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164336060517_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164336060517_00006005381.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164336215407_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164336215407_00006005389.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164336361845_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164336361845_00006005610.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164336502312_0000600

/data/darknet/python//croped_manufacture_images/20190603164344222288_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164344222288_00006005633.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164344388272_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164344388272_00006005710.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164344550274_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164344550274_00006005300.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164344726618_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164344726618_00006005381.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164344889901_00006005389.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164344889901_00006005389.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164345061533_00006005610.jpg
/data

(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164352052453_00006005610.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164352052453_00006005610.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164352209926_00006005633.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164352209926_00006005633.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164352372583_00006005710.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164352372583_00006005710.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164352533113_00006005300.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164352533113_00006005300.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164352709536_00006005381.jpg
/data/darknet/python//croped_manufacture_xmls/20190603164352709536_00006005381.xml
(469, 834, 3)
/data/darknet/python//croped_manufacture_images/20190603164352877602_0000600